http://cocodataset.org/#download

In [1]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
plt.style.use("ggplot")
%matplotlib inline

from tqdm import tqdm_notebook, tnrange
from itertools import chain
from skimage.io import imread, imshow, concatenate_images
from skimage.transform import resize
from skimage.morphology import label
from sklearn.model_selection import train_test_split

import tensorflow as tf

from cv2 import filter2D

from keras.models import Model, load_model
from keras.layers import Input, BatchNormalization, Activation, Dense, Dropout
from keras.layers.core import Lambda, RepeatVector, Reshape
from keras.layers.convolutional import Conv2D, Conv2DTranspose
from keras.layers.pooling import MaxPooling2D, GlobalMaxPool2D
from keras.layers.merge import concatenate, add
from keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau
from keras.optimizers import Adam
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

Using TensorFlow backend.


In [2]:
ids = next(os.walk("data/images"))[2] # list of names all images in the given path
ids_m = next(os.walk("data/masks"))[2] # list of names all images in the given path
# ids = ids[:1000]
# ids_m = ids_m[:1000]

print("No. of images = ", len(ids))

No. of images =  5100


In [96]:
im_width = 128
im_height = 128
border = 5

In [97]:
X = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)
y = np.zeros((len(ids), im_height, im_width, 1), dtype=np.float32)

In [98]:
kernel_size = 3
kernel = -1 * np.ones((kernel_size, kernel_size))
kernel[kernel_size // 2 + 1, kernel_size// 2 + 1] = kernel_size ** 2 - 1

for n, id_ in tqdm_notebook(enumerate(ids), total=len(ids)):
    img = load_img("data/images/"+id_, grayscale=True)
    x_img = img_to_array(img)
    x_img = resize(x_img, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
    # Load masks
    mask = img_to_array(load_img("data/masks/"+ id_[:-4] + '.png', color_mode = "grayscale"))
    mask = filter2D(mask[...,0] , -1, kernel)
    mask = resize(mask, (im_width, im_height, 1), mode = 'constant', preserve_range = True)
    mask[mask!=0] = 1
    X[n] = x_img/255.0
    y[n] = mask/255.0
    
X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=0.1, random_state=42)

In [6]:
def conv2d_block(input_tensor, n_filters, kernel_size=3, batchnorm=True):
    # first layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(input_tensor)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    # second layer
    x = Conv2D(filters=n_filters, kernel_size=(kernel_size, kernel_size), kernel_initializer="he_normal",
               padding="same")(x)
    if batchnorm:
        x = BatchNormalization()(x)
    x = Activation("relu")(x)
    return x

In [7]:
def get_unet(input_img, n_filters=16, dropout=0.5, batchnorm=True):
    # contracting path
    c1 = conv2d_block(input_img, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    p1 = MaxPooling2D((2, 2)) (c1)
    p1 = Dropout(dropout*0.5)(p1)

    c2 = conv2d_block(p1, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)
    p2 = MaxPooling2D((2, 2)) (c2)
    p2 = Dropout(dropout)(p2)

    c3 = conv2d_block(p2, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)
    p3 = MaxPooling2D((2, 2)) (c3)
    p3 = Dropout(dropout)(p3)

    c4 = conv2d_block(p3, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)
    p4 = MaxPooling2D(pool_size=(2, 2)) (c4)
    p4 = Dropout(dropout)(p4)
    
    c5 = conv2d_block(p4, n_filters=n_filters*16, kernel_size=3, batchnorm=batchnorm)
    
    # expansive path
    u6 = Conv2DTranspose(n_filters*8, (3, 3), strides=(2, 2), padding='same') (c5)
    u6 = concatenate([u6, c4])
    u6 = Dropout(dropout)(u6)
    c6 = conv2d_block(u6, n_filters=n_filters*8, kernel_size=3, batchnorm=batchnorm)

    u7 = Conv2DTranspose(n_filters*4, (3, 3), strides=(2, 2), padding='same') (c6)
    u7 = concatenate([u7, c3])
    u7 = Dropout(dropout)(u7)
    c7 = conv2d_block(u7, n_filters=n_filters*4, kernel_size=3, batchnorm=batchnorm)

    u8 = Conv2DTranspose(n_filters*2, (3, 3), strides=(2, 2), padding='same') (c7)
    u8 = concatenate([u8, c2])
    u8 = Dropout(dropout)(u8)
    c8 = conv2d_block(u8, n_filters=n_filters*2, kernel_size=3, batchnorm=batchnorm)

    u9 = Conv2DTranspose(n_filters*1, (3, 3), strides=(2, 2), padding='same') (c8)
    u9 = concatenate([u9, c1], axis=3)
    u9 = Dropout(dropout)(u9)
    c9 = conv2d_block(u9, n_filters=n_filters*1, kernel_size=3, batchnorm=batchnorm)
    
    outputs = Conv2D(1, (1, 1), activation='sigmoid') (c9)
    model = Model(inputs=[input_img], outputs=[outputs])
    return model

In [38]:
from keras.layers import Layer
import keras.backend as K

In [122]:
def weighted_bce(y_true, y_pred):
    weights = (y_true * 2.) + 1.
    bce = K.binary_crossentropy(y_true, y_pred)
    weighted_bce = K.mean(bce * weights)
    return weighted_bce

In [123]:
input_img = Input((im_height, im_width, 1), name='img')
model = get_unet(input_img, n_filters=16, dropout=0.01, batchnorm=True)

# model.compile(optimizer=Adam(), loss='binary_crossentropy', metrics=["accuracy"])
model.compile(optimizer=Adam(), loss=weighted_bce, metrics=["accuracy"])

# model.summary()

In [124]:
callbacks = [
    EarlyStopping(patience=10, verbose=0),
    ReduceLROnPlateau(factor=0.1, patience=3, min_lr=0.00001, verbose=0),
    ModelCheckpoint('model_weighted_1.h5', verbose=1, save_best_only=True, save_weights_only=False)
]


In [125]:
results = model.fit(X_train, y_train, batch_size=20, epochs=3, callbacks=callbacks,
                    validation_data=(X_valid, y_valid), class_weight = 'auto')


Train on 4590 samples, validate on 510 samples
Epoch 1/3


3680/4590 [=======================>......] - ETA: 22:34 - loss: 1.0496 - accuracy: 0.189 - ETA: 12:27 - loss: 1.0276 - accuracy: 0.195 - ETA: 9:04 - loss: 1.0059 - accuracy: 0.194 - ETA: 7:23 - loss: 0.9859 - accuracy: 0.19 - ETA: 6:21 - loss: 0.9608 - accuracy: 0.20 - ETA: 5:40 - loss: 0.9359 - accuracy: 0.22 - ETA: 5:10 - loss: 0.9121 - accuracy: 0.24 - ETA: 4:48 - loss: 0.8879 - accuracy: 0.26 - ETA: 4:30 - loss: 0.8654 - accuracy: 0.28 - ETA: 4:16 - loss: 0.8450 - accuracy: 0.30 - ETA: 4:05 - loss: 0.8260 - accuracy: 0.32 - ETA: 3:55 - loss: 0.8074 - accuracy: 0.34 - ETA: 3:46 - loss: 0.7974 - accuracy: 0.35 - ETA: 3:39 - loss: 0.7860 - accuracy: 0.37 - ETA: 3:33 - loss: 0.7742 - accuracy: 0.38 - ETA: 3:27 - loss: 0.7626 - accuracy: 0.39 - ETA: 3:22 - loss: 0.7494 - accuracy: 0.41 - ETA: 3:17 - loss: 0.7377 - accuracy: 0.42 - ETA: 3:13 - loss: 0.7265 - accuracy: 0.44 - ETA: 3:09 - loss: 0.7162 - accuracy: 0.45 - ETA: 3:06 - loss: 0.7064 - accuracy: 0.46 - ETA: 3:03 - loss: 0.6966 -

4590/4590 [==============================] - ETA: 30s - loss: 0.3035 - accuracy: 0.722 - ETA: 29s - loss: 0.3026 - accuracy: 0.722 - ETA: 29s - loss: 0.3017 - accuracy: 0.722 - ETA: 28s - loss: 0.3008 - accuracy: 0.722 - ETA: 27s - loss: 0.2999 - accuracy: 0.722 - ETA: 27s - loss: 0.2990 - accuracy: 0.723 - ETA: 26s - loss: 0.2982 - accuracy: 0.723 - ETA: 25s - loss: 0.2973 - accuracy: 0.723 - ETA: 25s - loss: 0.2965 - accuracy: 0.723 - ETA: 24s - loss: 0.2956 - accuracy: 0.723 - ETA: 23s - loss: 0.2947 - accuracy: 0.723 - ETA: 23s - loss: 0.2939 - accuracy: 0.723 - ETA: 22s - loss: 0.2931 - accuracy: 0.723 - ETA: 21s - loss: 0.2922 - accuracy: 0.723 - ETA: 21s - loss: 0.2914 - accuracy: 0.723 - ETA: 20s - loss: 0.2906 - accuracy: 0.724 - ETA: 19s - loss: 0.2897 - accuracy: 0.724 - ETA: 19s - loss: 0.2889 - accuracy: 0.724 - ETA: 18s - loss: 0.2881 - accuracy: 0.724 - ETA: 17s - loss: 0.2873 - accuracy: 0.724 - ETA: 16s - loss: 0.2865 - accuracy: 0.725 - ETA: 16s - loss: 0.2857 - accur

3720/4590 [=======================>......] - ETA: 3:00 - loss: 0.1090 - accuracy: 0.80 - ETA: 3:04 - loss: 0.1099 - accuracy: 0.78 - ETA: 3:06 - loss: 0.1102 - accuracy: 0.78 - ETA: 3:06 - loss: 0.1098 - accuracy: 0.77 - ETA: 3:06 - loss: 0.1097 - accuracy: 0.76 - ETA: 3:04 - loss: 0.1092 - accuracy: 0.77 - ETA: 3:02 - loss: 0.1088 - accuracy: 0.76 - ETA: 3:02 - loss: 0.1084 - accuracy: 0.76 - ETA: 3:06 - loss: 0.1081 - accuracy: 0.76 - ETA: 3:07 - loss: 0.1079 - accuracy: 0.76 - ETA: 3:07 - loss: 0.1076 - accuracy: 0.76 - ETA: 3:07 - loss: 0.1072 - accuracy: 0.76 - ETA: 3:06 - loss: 0.1069 - accuracy: 0.76 - ETA: 3:04 - loss: 0.1067 - accuracy: 0.76 - ETA: 3:03 - loss: 0.1064 - accuracy: 0.76 - ETA: 3:03 - loss: 0.1061 - accuracy: 0.76 - ETA: 3:02 - loss: 0.1058 - accuracy: 0.76 - ETA: 3:01 - loss: 0.1055 - accuracy: 0.76 - ETA: 3:00 - loss: 0.1053 - accuracy: 0.76 - ETA: 2:59 - loss: 0.1050 - accuracy: 0.76 - ETA: 2:58 - loss: 0.1047 - accuracy: 0.76 - ETA: 2:59 - loss: 0.1044 - accu

4590/4590 [==============================] - ETA: 36s - loss: 0.0748 - accuracy: 0.760 - ETA: 35s - loss: 0.0746 - accuracy: 0.760 - ETA: 34s - loss: 0.0745 - accuracy: 0.760 - ETA: 33s - loss: 0.0744 - accuracy: 0.760 - ETA: 32s - loss: 0.0742 - accuracy: 0.759 - ETA: 31s - loss: 0.0741 - accuracy: 0.759 - ETA: 30s - loss: 0.0740 - accuracy: 0.759 - ETA: 30s - loss: 0.0739 - accuracy: 0.759 - ETA: 29s - loss: 0.0737 - accuracy: 0.759 - ETA: 28s - loss: 0.0736 - accuracy: 0.759 - ETA: 27s - loss: 0.0735 - accuracy: 0.759 - ETA: 26s - loss: 0.0734 - accuracy: 0.759 - ETA: 25s - loss: 0.0732 - accuracy: 0.759 - ETA: 24s - loss: 0.0731 - accuracy: 0.759 - ETA: 24s - loss: 0.0730 - accuracy: 0.760 - ETA: 23s - loss: 0.0729 - accuracy: 0.760 - ETA: 22s - loss: 0.0727 - accuracy: 0.760 - ETA: 21s - loss: 0.0726 - accuracy: 0.760 - ETA: 20s - loss: 0.0725 - accuracy: 0.759 - ETA: 19s - loss: 0.0724 - accuracy: 0.759 - ETA: 18s - loss: 0.0722 - accuracy: 0.759 - ETA: 18s - loss: 0.0721 - accur

3720/4590 [=======================>......] - ETA: 2:39 - loss: 0.0434 - accuracy: 0.76 - ETA: 2:45 - loss: 0.0431 - accuracy: 0.77 - ETA: 2:47 - loss: 0.0431 - accuracy: 0.77 - ETA: 2:45 - loss: 0.0431 - accuracy: 0.76 - ETA: 2:44 - loss: 0.0430 - accuracy: 0.77 - ETA: 2:44 - loss: 0.0429 - accuracy: 0.77 - ETA: 2:42 - loss: 0.0429 - accuracy: 0.77 - ETA: 2:42 - loss: 0.0429 - accuracy: 0.76 - ETA: 2:41 - loss: 0.0428 - accuracy: 0.76 - ETA: 2:39 - loss: 0.0428 - accuracy: 0.76 - ETA: 2:39 - loss: 0.0428 - accuracy: 0.76 - ETA: 2:38 - loss: 0.0427 - accuracy: 0.76 - ETA: 2:37 - loss: 0.0426 - accuracy: 0.76 - ETA: 2:36 - loss: 0.0426 - accuracy: 0.76 - ETA: 2:35 - loss: 0.0425 - accuracy: 0.76 - ETA: 2:35 - loss: 0.0424 - accuracy: 0.76 - ETA: 2:34 - loss: 0.0424 - accuracy: 0.76 - ETA: 2:33 - loss: 0.0423 - accuracy: 0.76 - ETA: 2:32 - loss: 0.0423 - accuracy: 0.75 - ETA: 2:32 - loss: 0.0422 - accuracy: 0.75 - ETA: 2:31 - loss: 0.0421 - accuracy: 0.75 - ETA: 2:31 - loss: 0.0421 - accu

4590/4590 [==============================] - ETA: 30s - loss: 0.0335 - accuracy: 0.758 - ETA: 30s - loss: 0.0334 - accuracy: 0.758 - ETA: 29s - loss: 0.0334 - accuracy: 0.758 - ETA: 28s - loss: 0.0334 - accuracy: 0.758 - ETA: 27s - loss: 0.0333 - accuracy: 0.758 - ETA: 27s - loss: 0.0333 - accuracy: 0.758 - ETA: 26s - loss: 0.0332 - accuracy: 0.758 - ETA: 25s - loss: 0.0332 - accuracy: 0.758 - ETA: 24s - loss: 0.0331 - accuracy: 0.758 - ETA: 24s - loss: 0.0331 - accuracy: 0.758 - ETA: 23s - loss: 0.0331 - accuracy: 0.758 - ETA: 22s - loss: 0.0330 - accuracy: 0.758 - ETA: 22s - loss: 0.0330 - accuracy: 0.758 - ETA: 21s - loss: 0.0330 - accuracy: 0.758 - ETA: 20s - loss: 0.0329 - accuracy: 0.758 - ETA: 19s - loss: 0.0329 - accuracy: 0.758 - ETA: 19s - loss: 0.0328 - accuracy: 0.758 - ETA: 18s - loss: 0.0328 - accuracy: 0.758 - ETA: 17s - loss: 0.0328 - accuracy: 0.758 - ETA: 16s - loss: 0.0327 - accuracy: 0.758 - ETA: 16s - loss: 0.0327 - accuracy: 0.758 - ETA: 15s - loss: 0.0326 - accur